# Run experiment multi output regression


---


The aim of this script is to run the experiment in order to train a multioutput regression neural network to predict the size of cell parameters of different crystal structures. Users have to load the crystal structure dataset, read it, select which are the features to be used to train the model, as well as, which are the response variables. 

After that, dataset will be splitted in 80% for training and 20% for test dataset. Repeated K-fold cross validation will be used with *k* and *n* equals to 10. Hyperparameters optimization will be also used. 

## Import library

In [ ]:
import os

# Data Manipulation
import numpy as np
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

## Load the Data


---


User must enter the name of the path in which dataset is stored. After that, we will check if the directory exists and if it is empty or not.

### *Check directory and files*

In [ ]:
def read_dataset ():
  dataset = ''
  path_name = input('Enter the path name for dataset: ')
  path_name = '/content/' + path_name

  if not os.path.exists(path_name):
      print('Error! Invalid path selected.')
  else:
      print(path_name + ' is a valid path.')

      if not os.listdir(path_name):
        print("Warning! Empty directory.")
      else:
        file_name = input('Enter the file name for dataset: ')
        dataset = pd.read_csv(path_name + '/' + file_name + '.csv', sep = ';', index_col = 'ID_Observations' )
  return dataset

### *Load the dataset*

In [ ]:
y_coord_dataset = read_dataset()

In [ ]:
print('y_coord_dataset shape: {}'.format(y_coord_dataset.shape))
print('\n data types: \n{}'.format(y_coord_dataset.dtypes))
print('\ny_coord_dataset content: \n')
y_coord_dataset

## Insert details on dataset features


---


Users have to specify which variables will be used as features and which will be response features.

In [ ]:
def insert_dataset_structure_details(final_dataset):
    print('============================================================================')
    print('Insert dataset structure details');

    while 1:
        print('----------------------------------------------------------------------------')
        use_volume = input('\n1) Do you want to use volume as a feature in the experiment? [Y|N]: ');
        if use_volume.lower() == 'y':
          use_volume = True;
          break;
        elif use_volume.lower() == 'n':
          use_volume = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        use_total_n_peaks = input('\n2) Do you want to use total_n_peaks as a feature in the experiment? [Y|N]: ');
        if use_total_n_peaks.lower() == 'y':
          use_total_n_peaks = True;
          break;
        elif use_total_n_peaks.lower() == 'n':
          use_total_n_peaks = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        use_max_peaks = input('\n3) Do you want to use max_peaks as a feature in the experiment? [Y|N]: ');
        if use_max_peaks.lower() == 'y':
          use_max_peaks = True;
          break;
        elif use_max_peaks.lower() == 'n':
          use_max_peaks = False;
          break;
   
    while 1:
        print('----------------------------------------------------------------------------')
        a_is_response = input('\n4) "a" is a response feature ? [Y|N]: ');
        if a_is_response.lower() == 'y':
          a_is_response = True;
          break;
        elif a_is_response.lower() == 'n':
          a_is_response = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        b_is_response = input('\n5) "b" is a response feature ? [Y|N]: ');
        if b_is_response.lower() == 'y':
          b_is_response = True;
          break;
        elif b_is_response.lower() == 'n':
          b_is_response = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        c_is_response = input('\n6) "c" is a response feature ? [Y|N]: ');
        if c_is_response.lower() == 'y':
          c_is_response = True;
          break;
        elif c_is_response.lower() == 'n':
          c_is_response = False;
          break;

    print('============================================================================')

    response_features_name = []

    if not use_total_n_peaks:
      final_dataset.drop('Total_n_peaks', axis=1, inplace=True)
    
    if not use_max_peaks:
      final_dataset.drop('Max_peaks_position', axis=1, inplace=True)

    if not use_volume:
      final_dataset.drop('Volume', axis=1, inplace=True)
    
    if not a_is_response:
      final_dataset.drop('a', axis=1, inplace=True)
    else:
      response_features_name.append('a')
    
    if not b_is_response:
      final_dataset.drop('b', axis=1, inplace=True)
    else:
      response_features_name.append('b')
    
    if not c_is_response:
      final_dataset.drop('c', axis=1, inplace=True)  
    else:
      response_features_name.append('c')

    final_dataset.drop('Crystal_Structure_Type', axis=1, inplace=True)

    return final_dataset, response_features_name

In [ ]:
final_dataset_for_experiment, response_features_name = insert_dataset_structure_details(y_coord_dataset.copy())
print('The response features for this crystal structure are: {}'.format(response_features_name))
print('The final dataset with features is:\n')
final_dataset_for_experiment

## Overview of the data

### *Descriptive Statistics*
Using the method describe() we can see some information about the dataset we have, we can have a picture of each column (feature):
*   Mean, mediam, model, standard deviation.
*   Min and Max.
*   Count.
*   Quartiles.








In [ ]:
final_dataset_for_experiment.describe()

### *Boxplot*

In [ ]:
def plot_boxplot(df, boxplot_title):
  fig = px.box(df, template='simple_white')
  fig.update_layout(title_text=boxplot_title,
                    xaxis_title='features',
                    yaxis_title='',
                    title_x=0.5, 
                    font=dict(
                        size=16
                    ))
  fig.show()

In [ ]:
plot_boxplot(final_dataset_for_experiment, 'Boxplot of dataset distribution')

### *Correlation matrix*

In [ ]:
def plot_correlation_matrix(df, corr_matrix_title):
  df_corr = df.corr()
  fig = go.Figure()
  fig.add_trace(
      go.Heatmap(
          x = df_corr.columns,
          y = df_corr.index,
          z = np.array(df_corr),
          text=df_corr.values,
          texttemplate='%{text:.2f}'
      )
  )
  fig.update_layout(title_text= corr_matrix_title,
                      xaxis_title='',
                      yaxis_title='',
                      title_x=0.5, 
                      font=dict(
                          size=16
                      ))
  fig.show()

In [ ]:
plot_correlation_matrix(final_dataset_for_experiment, 'Correlation matrix of dataset')

### *Pairplot*

In [ ]:
print('Pairplot of dataset')
sns.pairplot(final_dataset_for_experiment)

## Split data for regression task

### *Split final dataset in x (features) and y (response)*

In [ ]:
x = final_dataset_for_experiment[final_dataset_for_experiment.columns.difference(response_features_name)]
print('Features to be used for training are: ')
x

In [ ]:
y = final_dataset_for_experiment[response_features_name]
print('Response features are:')
y

### *Split dataset in training and test set*
We use 80% of the original dataset as training dataset and the remaining 20% to test the model.

**random_state messo per riproducibilità**
**da cancellare**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
print('Training dataset shape')
print('- The x_train dataframe has the following shape: {}'.format(x_train.shape))
print('- The y_train dataframe has the following shape: {}'.format(y_train.shape))
print('------------------------------------------------------------------------')
print('Test dataset shape')
print('- The x_test dataframe has the following shape: {}'.format(x_test.shape))
print('- The y_test dataframe has the following shape: {}'.format(y_test.shape))

## Building the multi-output regression neural network model